Extracting Review

In [4]:
#installing relevant libraries 
import time
import pandas as pd
import regex as re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager



#### Steam Scraping software

In [8]:
def SteamScraping(id, name, num_rev):

    # Set up the Selenium WebDriver (make sure you have the appropriate WebDriver)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Define the URL of the Steam reviews page
    url = f"https://steamcommunity.com/app/{id}/reviews/?browsefilter=toprated&snr=1_5_100010_"
    driver.get(url)

    # Wait for the page to load
    time.sleep(3)

    # Set to keep track of unique reviews
    game_title = []
    unique_reviews = []
    approvals = []
    helpfulCount = []
    CommentCount = []

    run = True

    # Infinite scrolling loop
    while run == True:

        for scrolls in range(20):
            # Scroll down to the bottom of the page
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # Wait for new reviews to load before scolling again
            time.sleep(2)

        # Get review elements
        reviews = driver.find_elements(By.CLASS_NAME, 'apphub_UserReviewCardContent ')
        ComSecs = driver.find_elements(By.CLASS_NAME, 'apphub_CardContentAuthorBlock.tall ')
        
        # Extract reviews text
        for review in reviews:

            #add review text to unique_reviews
            review_text = review.find_element(By.CLASS_NAME, 'apphub_CardTextContent').text.strip()
            unique_reviews.append(review_text)

            #add approve to approvals
            vote_header = review.find_element(By.CLASS_NAME, 'vote_header ')
            review_info = vote_header.find_element(By.CLASS_NAME, 'reviewInfo ')
            approve = review_info.find_element(By.CLASS_NAME, 'title').text.strip()
            approvals.append(approve)

            #add review text to unique_reviews
            help_funny = review.find_element(By.CLASS_NAME, 'found_helpful').text.strip()
            helpfulCount.append(help_funny)

            if len(unique_reviews) >= num_rev: 
                run = False

        # itterate through comment sections
        for ComSec in ComSecs:
            Coms = ComSec.find_element(By.CLASS_NAME, 'apphub_UserReviewCardStats')
            ComCount = Coms.find_element(By.CLASS_NAME, 'apphub_CardCommentCount.alignNews').text.strip()
            CommentCount.append(ComCount)
            if len(CommentCount) >= num_rev: 
                run = False

        for n in range(len(unique_reviews)): 
            game_title.append(name)
            
    print(name)
    print(len(game_title))
    # Print the unique reviews
    #for review in unique_reviews:

    # Close the WebDriver
    driver.quit()
    return(pd.DataFrame({'games':game_title,'review':unique_reviews,'approval':approvals,'helpCount':helpfulCount,'ComCount':CommentCount}))



In [9]:
#import GamesList
games = pd.read_csv("GamesList.csv")
games = games.set_index("id") 

#build a reviews data Frame
reviews_df = pd.DataFrame({'games':[],'review':[],'approval':[],'helpCount':[],'ComCount':[]})

#remove dlc for Project Winter - Blackout
games = games.drop(1168030)

for row in range(len(games)):
    #SteamScraping(games.index[row], games.title[games.index[row]], 3)
    reviews_df = pd.concat([reviews_df, SteamScraping(games.index[row], games.title[games.index[row]], 200)])

reviews_df.reset_index()
reviews_df

Snakeybus
210
Biped
210
Worms Rumble
210
Wobbly Life
210
Party Animals
210
Bloons TD Battles 2
210
PICO PARK
210
Perfect Heist 2
210
Goose Goose Duck
210
Stumble Guys
210
Bopl Battle
210
Crab Game
210
The Riflemen
210
Scribblenauts Unlimited
210
Pit People®
210
Duck Game
210
ShellShock Live
210
Worms W.M.D
210
Ratz Instagib
210
Cat Goes Fishing
210
The Mean Greens - Plastic Warfare
210
Tower Unite
210
Clustertruck
210
Garry's Mod
210
Who's Your Daddy?!
210
Human Fall Flat
210
Happy Room
210
Pikuniku
210
Stick Fight: The Game
210
Worms_Ultimate_Mayhem
210
Overcooked! 2
210
Project Winter
210
Super Animal Royale
210
Pummel Party
210
ibb & obb
210
Bloons TD 6
210


,games,review,approval,helpCount,ComCount
0,Snakeybus,"Posted: 31 December, 2019\nHave you ever wante...",Recommended,255 people found this review helpful\n121 peop...,0
1,Snakeybus,"Posted: 29 December, 2023\nhere to say hi to a...",Recommended,180 people found this review helpful\n15 peopl...,0
2,Snakeybus,"Posted: 13 February, 2021\nI bought this becau...",Recommended,99 people found this review helpful\n12 people...,2
3,Snakeybus,"Posted: 28 July, 2019\nIt's simple and fun for...",Recommended,76 people found this review helpful\n22 people...,0
4,Snakeybus,"Posted: 22 April, 2020\nbus",Recommended,149 people found this review helpful\n50 peopl...,4
...,...,...,...,...,...
205,Bloons TD 6,"Posted: August 12, 2021\nMONKE",Recommended,177 people found this review helpful\n55 peopl...,0
206,Bloons TD 6,"Posted: June 22, 2023\nTHIS IS THE MOST VIOLEN...",Recommended,122 people found this review helpful\n95 peopl...,4
207,Bloons TD 6,"Posted: May 17, 2022\nYes, I'm a simp\n\nS nip...",Recommended,109 people found this review helpful\n72 peopl...,0
208,Bloons TD 6,"Posted: January 11, 2023\nBloons TD 6 is a tow...",Recommended,83 people found this review helpful\n5 people ...,8


In [15]:
reviews_df.reset_index(inplace = True)



Export the Reviews to a csv

In [52]:
reviews_df.to_csv("ScrappedSteamReviews.csv")
